In [1]:
import numpy as np
import pandas as pd

from datascience import *

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

import locale

In [2]:
# Import CSV
csv_directory = "data/csv/"

all_sales_data = Table.read_table(csv_directory+'manhattan.csv', low_memory=False)


In [3]:
# remove and sales less than $100 they are strange


all_sales_data = all_sales_data.where('SALE PRICE', are.above(100000))

In [4]:
# SPOT CHECK: Understand the data lables we might be interested in
all_sales_data.sort("SALE PRICE").show(20)
all_sales_data.labels

Unnamed: 0,ADDRESS,APARTMENT NUMBER,BLOCK,BOROUGH,BUILDING CLASS AS OF FINAL ROLL 17/18,BUILDING CLASS AT PRESENT,BUILDING CLASS AT TIME OF SALE,BUILDING CLASS CATEGORY,COMMERCIAL UNITS,EASE-MENT,GROSS SQUARE FEET,LAND SQUARE FEET,LOT,NEIGHBORHOOD,RESIDENTIAL UNITS,SALE DATE,SALE PRICE,TAX CLASS AS OF FINAL ROLL 17/18,TAX CLASS AT PRESENT,TAX CLASS AT TIME OF SALE,TOTAL UNITS,YEAR BUILT,ZIP CODE
9562,102 WEST 57TH STREET,,1009,1,nan,H2,H2,25 LUXURY HOTELS,2,,112850,7532,37,MIDTOWN WEST,0,2011-10-19,100009,nan,4,4,2,2007,10019
10168,102 WEST 57TH STREET,,1009,1,nan,H2,H2,25 LUXURY HOTELS,2,,112850,7532,37,MIDTOWN WEST,0,2011-07-27,100009,nan,4,4,2,2007,10019
10547,90 LA SALLE ST,,1978,1,nan,D4,D4,10 COOPS - ELEVATOR APARTMENTS,0,,0,0,1,MORNINGSIDE HEIGHTS,0,2003-12-05,100033,nan,2,2,0,1956,10027
371,"365 WEST 25TH STREET, 14F",,749,1,nan,D4,D4,10 COOPS - ELEVATOR APARTMENTS,0,,0,0,1,CHELSEA,0,2016-08-01,100042,nan,2,2,0,1963,10001
372,"365 WEST 25TH STREET, 4A",,749,1,nan,D4,D4,10 COOPS - ELEVATOR APARTMENTS,0,,0,0,1,CHELSEA,0,2016-08-01,100042,nan,2,2,0,1963,10001
401,"345 8TH AVENUE, 3A",,751,1,nan,D4,D4,10 COOPS - ELEVATOR APARTMENTS,0,,0,0,1,CHELSEA,0,2016-08-01,100042,nan,2,2,0,1960,10001
12025,102 WEST 57TH STREET,,1009,1,nan,H2,H2,25 LUXURY HOTELS,2,,112850,7532,37,MIDTOWN WEST,0,2013-05-14,100060,nan,4,4,2,2007,10019
11787,102 WEST 57TH STREET,,1009,1,nan,H2,H2,25 LUXURY HOTELS,2,,112850,7532,37,MIDTOWN WEST,0,2013-07-02,100100,nan,4,4,2,2007,10019
11920,102 WEST 57TH STREET,,1009,1,nan,H2,H2,25 LUXURY HOTELS,2,,112850,7532,37,MIDTOWN WEST,0,2013-06-05,100100,nan,4,4,2,2007,10019
12277,102 WEST 57TH ST,,1009,1,nan,H2,H2,25 LUXURY HOTELS,2,,112850,7532,37,MIDTOWN WEST,0,2013-03-05,100100,nan,4,4,2,2007,10019


('Unnamed: 0',
 'ADDRESS',
 'APARTMENT NUMBER',
 'BLOCK',
 'BOROUGH',
 'BUILDING CLASS AS OF FINAL ROLL 17/18',
 'BUILDING CLASS AT PRESENT',
 'BUILDING CLASS AT TIME OF SALE',
 'BUILDING CLASS CATEGORY',
 'COMMERCIAL UNITS',
 'EASE-MENT',
 'GROSS SQUARE FEET',
 'LAND SQUARE FEET',
 'LOT',
 'NEIGHBORHOOD',
 'RESIDENTIAL UNITS',
 'SALE DATE',
 'SALE PRICE',
 'TAX CLASS AS OF FINAL ROLL 17/18',
 'TAX CLASS AT PRESENT',
 'TAX CLASS AT TIME OF SALE',
 'TOTAL UNITS',
 'YEAR BUILT',
 'ZIP CODE')

In [5]:
# Remove redundant cells

all_sales_data = all_sales_data.select(['SALE DATE', 'SALE PRICE', 'ADDRESS','APARTMENT NUMBER', 'YEAR BUILT', 'NEIGHBORHOOD', 'ZIP CODE', 'BUILDING CLASS AT TIME OF SALE', 'BUILDING CLASS CATEGORY'])

In [6]:
# Combine the Address and Apt No

def combine_address(address, aptNo):
    """Combine the address and Apartment into a single result"""
    temp = address.strip()
    if len(aptNo.strip()) > 0:
        temp = temp + ', ' + aptNo.strip()
    return temp

full_address = all_sales_data.apply(combine_address, ['ADDRESS', 'APARTMENT NUMBER'])

/usr/local/lib/python3.6/site-packages/datascience/tables.py:361: FutureWarning: column lists are deprecated; pass each as an argument
  "column lists are deprecated; pass each as an argument", FutureWarning)


In [7]:
# Add a Full Address column
all_sales_data =  all_sales_data.with_column('FULL ADDRESS', full_address)

In [8]:
# Try to understand Building Class Codes. Whare the most common Codes
all_sales_data.group(['BUILDING CLASS AT TIME OF SALE', 'BUILDING CLASS CATEGORY']).sort('count', descending=True).show(20)

# Question what's R5 -  COMMERCIAL CONDOS? ignoring for now

BUILDING CLASS AT TIME OF SALE,BUILDING CLASS CATEGORY,count
R4,13 CONDOS - ELEVATOR APARTMENTS,89835
D4,10 COOPS - ELEVATOR APARTMENTS,82040
R9,17 CONDOPS,9942
C6,09 COOPS - WALKUP APARTMENTS,9928
R4,13 CONDOS - ELEVATOR APARTMENTS,5852
D4,10 COOPS - ELEVATOR APARTMENTS,5286
R9,17 CONDO COOPS,4088
D0,10 COOPS - ELEVATOR APARTMENTS,3296
R1,15 CONDOS - 2-10 UNIT RESIDENTIAL,3247
R5,28 COMMERCIAL CONDOS,3228


In [9]:
# Let's pick out the condos for now. 
# See ## for condo codes
condos = all_sales_data.where('BUILDING CLASS AT TIME OF SALE', are.contained_in("R1R2R3R4R6"))

In [10]:
# Spot Check condo data
condos.sample(10)

SALE DATE,SALE PRICE,ADDRESS,APARTMENT NUMBER,YEAR BUILT,NEIGHBORHOOD,ZIP CODE,BUILDING CLASS AT TIME OF SALE,BUILDING CLASS CATEGORY,FULL ADDRESS
2005-06-08,1675000,203 WEST 90TH STREET,8A,0,UPPER WEST SIDE (79-96),10024,R4,13 CONDOS - ELEVATOR APARTMENTS,"203 WEST 90TH STREET, 8A"
2003-12-09,1410276,404 EAST 76 STREET,30C,0,UPPER EAST SIDE (59-79),10021,R4,13 CONDOS - ELEVATOR APARTMENTS,"404 EAST 76 STREET, 30C"
2016-01-21,1122620,5 FRANKLIN PLACE,2B,1920,CIVIC CENTER,10013,R4,13 CONDOS - ELEVATOR APARTMENTS,"5 FRANKLIN PLACE, 2B"
2017-12-07,2650000,170 EAST 87TH STREET,W21B,0,UPPER EAST SIDE (79-96),10128,R4,13 CONDOS - ELEVATOR APARTMENTS,"170 EAST 87TH STREET, W21B"
2007-08-15,820000,510 EAST 80TH STREET,7C,1986,UPPER EAST SIDE (79-96),10021,R4,13 CONDOS - ELEVATOR APARTMENTS,"510 EAST 80TH STREET, 7C"
2017-10-18,900000,225 WEST 60TH STREET,15A,2005,UPPER WEST SIDE (59-79),10023,R4,13 CONDOS - ELEVATOR APARTMENTS,"225 WEST 60TH STREET, 15A"
2006-05-25,1537557,425 FIFTH AVENUE,64A,0,MURRAY HILL,10016,R4,13 CONDOS - ELEVATOR APARTMENTS,"425 FIFTH AVENUE, 64A"
2012-10-19,575000,400 CENTRAL PARK WEST,15S,1960,MANHATTAN VALLEY,10025,R4,13 CONDOS - ELEVATOR APARTMENTS,"400 CENTRAL PARK WEST, 15S"
2015-05-05,9250000,530 PARK AVENUE,18A,1941,UPPER EAST SIDE (59-79),10065,R4,13 CONDOS - ELEVATOR APARTMENTS,"530 PARK AVENUE, 18A"
2015-09-15,2610000,1474 THIRD AVENUE,2S,0,UPPER EAST SIDE (79-96),10028,R4,13 CONDOS - ELEVATOR APARTMENTS,"1474 THIRD AVENUE, 2S"


In [11]:
# Let's find condos with multiple sales

condos.group('FULL ADDRESS').sort(1, descending=True).show(30)

# Looks like Apt numbers are missing for most of the top sales. let's ignore anything without an apt number. 

FULL ADDRESS,count
1 WEST END AVENUE,136
45 EAST 22ND STREET,51
245 WEST 99TH STREET,49
171 WEST 131 STREET,44
959 1 AVENUE,42
416 WEST 52ND STREET,39
245 WEST 99TH STREET,37
2094 FREDRICK DOUGLASS BL,35
172 MADISON AVENUE,35
360 EAST 89TH STREET,23


In [12]:
# Find condos with apt numbers i.e. clean data, then find those with more that 1 sale record.

multi_sale_condos = condos.where('FULL ADDRESS', are.containing(',')).group('FULL ADDRESS').sort(1, descending=True).where('count', are.above(1))
multi_sale_condos

#ahh much better, fewer sales but still roughly 15K records

FULL ADDRESS,count
"2 EAST 55 STREET, 835",8
"2 EAST 55TH STREET, 935",7
"2 EAST 55 STREET, 921",7
"77 PARK AVENUE, 2F",6
"418 CENTRAL PARK WEST, 22",6
"2 EAST 55 STREET, 936",6
"2 EAST 55 STREET, 903",6
"15 BROAD STREET, 2116",6
"140 7 AVENUE, 6A",6
"52 PARK AVENUE, 7",5


In [13]:
# Define condos with multiple sales. 

multi_sale_condos = multi_sale_condos.join('FULL ADDRESS', condos, 'FULL ADDRESS')
multi_sale_condos.sort(0)

FULL ADDRESS,count,SALE DATE,SALE PRICE,ADDRESS,APARTMENT NUMBER,YEAR BUILT,NEIGHBORHOOD,ZIP CODE,BUILDING CLASS AT TIME OF SALE,BUILDING CLASS CATEGORY
"1 BOND STREET, 2C",2,2017-08-18,5640000,1 BOND STREET,2C,0,GREENWICH VILLAGE-CENTRAL,10012,R4,13 CONDOS - ELEVATOR APARTMENTS
"1 BOND STREET, 2C",2,2010-12-15,600000,1 BOND STREET,2C,0,GREENWICH VILLAGE-CENTRAL,10012,R4,13 CONDOS - ELEVATOR APARTMENTS
"1 BOND STREET, 2D",2,2009-04-24,1195000,1 BOND STREET,2D,0,GREENWICH VILLAGE-CENTRAL,10012,R4,13 CONDOS - ELEVATOR APARTMENTS
"1 BOND STREET, 2D",2,2014-01-07,1925000,1 BOND STREET,2D,0,GREENWICH VILLAGE-CENTRAL,10012,R4,13 CONDOS - ELEVATOR APARTMENTS
"1 BOND STREET, 3A",2,2013-06-14,7200000,1 BOND STREET,3A,0,GREENWICH VILLAGE-CENTRAL,10012,R4,13 CONDOS - ELEVATOR APARTMENTS
"1 BOND STREET, 3A",2,2008-06-06,5030000,1 BOND STREET,3A,0,GREENWICH VILLAGE-CENTRAL,10012,R4,13 CONDOS - ELEVATOR APARTMENTS
"1 BOND STREET, 4D",2,2006-08-30,1565000,1 BOND STREET,4D,0,GREENWICH VILLAGE-CENTRAL,10012,R4,13 CONDOS - ELEVATOR APARTMENTS
"1 BOND STREET, 4D",2,2013-08-20,2415000,1 BOND STREET,4D,0,GREENWICH VILLAGE-CENTRAL,10012,R4,13 CONDOS - ELEVATOR APARTMENTS
"1 BOND STREET, 6A",2,2012-12-12,3600000,1 BOND STREET,6A,0,GREENWICH VILLAGE-CENTRAL,10012,R4,13 CONDOS - ELEVATOR APARTMENTS
"1 BOND STREET, 6A",2,2017-09-06,5250000,1 BOND STREET,6A,0,GREENWICH VILLAGE-CENTRAL,10012,R4,13 CONDOS - ELEVATOR APARTMENTS


In [14]:
# Find the first and last Sale Dates

first_sale_dates = multi_sale_condos.select(0, 2).group([0], min)
last_sale_dates = multi_sale_condos.select(0, 2).group([0], max)

# Spot check data
first_sale_dates.show(5)
last_sale_dates.show(5)

FULL ADDRESS,SALE DATE min
"1 BOND STREET, 2C",2010-12-15
"1 BOND STREET, 2D",2009-04-24
"1 BOND STREET, 3A",2008-06-06
"1 BOND STREET, 4D",2006-08-30
"1 BOND STREET, 6A",2012-12-12


FULL ADDRESS,SALE DATE max
"1 BOND STREET, 2C",2017-08-18
"1 BOND STREET, 2D",2014-01-07
"1 BOND STREET, 3A",2013-06-14
"1 BOND STREET, 4D",2013-08-20
"1 BOND STREET, 6A",2017-09-06


In [15]:
# Update Labels

first_sale_dates = first_sale_dates.relabel(1, 'FIRST SALE')
last_sale_dates = last_sale_dates.relabel(1, 'LAST SALE')

In [16]:
# Join with Condos to get the sale price
first_sale_dates = first_sale_dates.join("FULL ADDRESS", condos, 'FULL ADDRESS').where('SALE DATE', are.equal_to, 'FIRST SALE')
first_sale_dates = first_sale_dates.select('FULL ADDRESS', 'FIRST SALE', 'SALE PRICE').relabel(2, 'FIRST PRICE')


last_sale_dates = last_sale_dates.join("FULL ADDRESS", condos, 'FULL ADDRESS').where('SALE DATE', are.equal_to, 'LAST SALE')
last_sale_dates = last_sale_dates.select('FULL ADDRESS', 'LAST SALE', 'SALE PRICE').relabel(2, 'LAST PRICE')

In [17]:
first_sale_dates.show(5)
last_sale_dates.show(5)

# Hmm earlier we had 14730 now we have more and they don't match. Could we have duplicates??

FULL ADDRESS,FIRST SALE,FIRST PRICE
"1 BOND STREET, 2C",2010-12-15,600000
"1 BOND STREET, 2D",2009-04-24,1195000
"1 BOND STREET, 3A",2008-06-06,5030000
"1 BOND STREET, 4D",2006-08-30,1565000
"1 BOND STREET, 6A",2012-12-12,3600000


FULL ADDRESS,LAST SALE,LAST PRICE
"1 BOND STREET, 2C",2017-08-18,5640000
"1 BOND STREET, 2D",2014-01-07,1925000
"1 BOND STREET, 3A",2013-06-14,7200000
"1 BOND STREET, 4D",2013-08-20,2415000
"1 BOND STREET, 6A",2017-09-06,5250000


In [46]:
# Yep duplicate sales on the same day wtf!
first_sale_dates.where(0, are.containing('201 WEST 17TH')).show()
# first_sale_dates.groups([0,1]).sort(2, descending=True)

FULL ADDRESS,FIRST SALE,FIRST PRICE
"201 WEST 17TH STREET, 7A",2014-03-31,999652
"201 WEST 17TH STREET, 7A",2014-03-31,999652
"201 WEST 17TH STREET, 7A",2014-03-31,999652


In [25]:
condos.where('FULL ADDRESS', are.equal_to('2 EAST 55 STREET, 921')).show()
first_sale_dates.where('FULL ADDRESS', are.containing('201 WEST 17TH STREET'))

SALE DATE,SALE PRICE,ADDRESS,APARTMENT NUMBER,YEAR BUILT,NEIGHBORHOOD,ZIP CODE,BUILDING CLASS AT TIME OF SALE,BUILDING CLASS CATEGORY,FULL ADDRESS
2006-10-09,467251,2 EAST 55 STREET,921,1903,MIDTOWN CBD,10022,R4,13 CONDOS - ELEVATOR APARTMENTS,"2 EAST 55 STREET, 921"
2006-10-09,403977,2 EAST 55 STREET,921,1903,MIDTOWN CBD,10022,R4,13 CONDOS - ELEVATOR APARTMENTS,"2 EAST 55 STREET, 921"
2006-09-05,451230,2 EAST 55 STREET,921,1903,MIDTOWN CBD,10022,R4,13 CONDOS - ELEVATOR APARTMENTS,"2 EAST 55 STREET, 921"
2006-08-22,486720,2 EAST 55 STREET,921,1903,MIDTOWN CBD,10022,R4,13 CONDOS - ELEVATOR APARTMENTS,"2 EAST 55 STREET, 921"
2006-08-22,496860,2 EAST 55 STREET,921,1903,MIDTOWN CBD,10022,R4,13 CONDOS - ELEVATOR APARTMENTS,"2 EAST 55 STREET, 921"
2006-08-22,486922,2 EAST 55 STREET,921,1903,MIDTOWN CBD,10022,R4,13 CONDOS - ELEVATOR APARTMENTS,"2 EAST 55 STREET, 921"
2006-08-22,412393,2 EAST 55 STREET,921,1903,MIDTOWN CBD,10022,R4,13 CONDOS - ELEVATOR APARTMENTS,"2 EAST 55 STREET, 921"


FULL ADDRESS,FIRST SALE,FIRST PRICE


SALE DATE,SALE PRICE,ADDRESS,APARTMENT NUMBER,YEAR BUILT,NEIGHBORHOOD,ZIP CODE,BUILDING CLASS AT TIME OF SALE,BUILDING CLASS CATEGORY,FULL ADDRESS
2016-01-26,1934675,201 WEST 17TH STREET,5C,0,CHELSEA,10011,R4,13 CONDOS - ELEVATOR APARTMENTS,"201 WEST 17TH STREET, 5C"
2016-11-04,5250000,201 WEST 17TH STREET,PH-F,0,CHELSEA,10011,R4,13 CONDOS - ELEVATOR APARTMENTS,"201 WEST 17TH STREET, PH-F"
2004-12-01,1350000,201 WEST 17TH STREET,4C,0,CHELSEA,10011,R4,13 CONDOS - ELEVATOR APARTMENTS,"201 WEST 17TH STREET, 4C"
2005-06-20,2200000,201 WEST 17TH STREET,9B,0,CHELSEA,10011,R4,13 CONDOS - ELEVATOR APARTMENTS,"201 WEST 17TH STREET, 9B"
2010-04-23,2100000,201 WEST 17TH STREET,2B,0,CHELSEA,10011,R4,13 CONDOS - ELEVATOR APARTMENTS,"201 WEST 17TH STREET, 2B"
